In [97]:
import pandas as pd
import nltk
import re
import string
from collections import Counter

In [32]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [5]:
from parse_url import *
from mapping_urls_make import *

In [134]:
ingredientsKB = {}

ingredientsKB['dairyIngredients'] = ['buttermilk', 'cottage', 'cream', 'creamer', 'creamy', 'creme', 'ghee', 'half-and-half', 'milk', 'yogurt']
ingredientsKB['cheeses'] = ['bocconcini', 'mozzarella', 'gouda', 'swiss', 'brie']
ingredientsKB['meats'] = ['bacon', 'beefs', 'burgers', 'chorizo', 'dogs', 'frankfurters', 'giblets', 'ham', 'lambs', 'livers', 'meatballs', 'meatloaves', 'meats', 'mignon', 'mincemeat', 'pepperonis', "pig's", 'porks', 'prosciutto', 'ribs', 'roasts', 'sausages', 'sirloin', 'tripe', 'veal', 'venison', 'kielbasas', 'liverwurst', 'wieners', 'cotechino', 'linguica', 'pastrami', 'squirrels', 'sauerbraten', 'picadillo', 'carcass', 'brains', 'mortadella', 'rounds', 'sweetbread', 'toad', 'tinga', 'embutido', 'hash', 'broil', 'brisket', 'franks', 'pigs', 'rouladen', 'chops', 'scrapple', 'barbeque', 'spareribs']
ingredientsKB['poultry'] = ['bologna', 'bratwursts', 'chickens', 'ducks', 'goose', 'hens', 'pollo', 'salami', 'turkey', 'pheasant', 'quail', 'turducken', 'drumettes', 'wings', 'roosters']
ingredientsKB['fish'] = ['albacores', 'bass', 'catfish', 'cods', 'fish', 'flounder', 'grouper', 'haddock', 'halibut', 'mahi', 'monkfish', 'salmon', 'shark', 'snapper', 'sole', 'swordfishes', 'trouts', 'tunas', 'bluefish', 'bonito', 'rockfish', 'mackerel', 'naruto', 'drum', 'marlin', 'tilapia', 'carp', 'kingfish', 'mullets', 'whitefish', 'kippers', 'torsk', 'saltfish']
ingredientsKB['seafoods'] = ['anchovies', 'calamaris', 'clams', 'crabs', 'crabmeat', 'crawfish', 'lobsters', 'mussels', 'oysters', 'prawns', 'scallops', 'seafood', 'shrimps', 'squids', 'snails', 'shellfish', 'caviar']
ingredientsKB['mainProteins'] = ['beans', 'chickpeas', 'nuts', 'seeds', 'tofu', 'whey', 'buckwheat', 'protein', 'soybeans', 'soy', 'tempeh', 'lentils', 'masoor', 'gluten', 'pine', 'falafel', 'portobello']
ingredientsKB['fruits'] = ['apples', 'apricots', 'bananas', 'blackberries', 'blueberries', 'cantaloupe', 'cherries', 'citrons', 'citrus', 'coconuts', 'cranberries', 'currants', 'elderberries', 'figs', 'fruitcakes', 'fruits', 'gooseberries', 'grapefruit', 'grapes', 'guava', 'honeydew', 'huckleberries', 'kiwis','kumquats', 'lemonade', 'lemons', 'limes', 'mangoes', 'marrons', 'mincemeat', 'mulberries', 'nectarines', 'oranges', 'papayas', 'peaches', 'pears', 'persimmon', 'persimmons', 'pineapples', 'plums', 'prunes', 'raisins',  'raspberries', 'slushies', 'smoothies', 'sorrel', 'strawberries', 'tangerines', 'watermelons', 'yuzu', 'lingonberries', 'plantains', 'juniper', 'lingonberries', 'pomegranates', 'serviceberries', 'zinfandel', 'lychees', 'carambola', 'uvas']
ingredientsKB['vegetables'] = ['artichokes', 'arugula', 'asparagus', 'avocados', 'bamboo', 'beets', 'broccoli', 'cabbage', 'calzones', 'carrots', 'cauliflower', 'celery', 'chilis', 'chives', 'choy', 'cilantro', 'coleslaw', 'coriander', 'cucumber', 'cucumbers', 'dates', 'eggplant', 'eggplants', 'endive', 'escarole', 'galangal', 'haystacks', 'jicama', 'kale', 'kohlrabi', 'kucai', 'leeks', 'lettuce', 'mushrooms', 'okra', 'olives', 'onions', 'parsley', 'parsnips', 'peas', 'peppers', 'pickles', 'pizzas', 'potatoes', 'pumpkins', 'radishes', 'rutabagas', 'salad', 'sauerkraut', 'shallots', 'slaws', 'spinach', 'sprouts', 'squash', 'tamarind', 'taros', 'tomatillo', 'tomatillos', 'tomatoes', 'turnips', 'vegetable', 'vegetables', 'veggies', 'watercress', 'yams', 'zucchinis', 'chervil', 'daikon', 'iceberg', 'nopales', 'pimentos', 'radicchio', 'karengo', 'nori', 'succotash', 'truffle', 'chard', 'fries', 'leaves', 'browns', 'romain', 'palm', 'sorghum', 'aloo', 'haricots', 'caprese', 'salata', 'shiitake']
ingredientsKB['sugars'] = ['Jell-O', 'butterscotch', 'candied', 'candy', 'caramels', 'frosting', 'fructose', 'gingersnaps', 'glaces', 'glaze', 'glycerin', 'glycerol', 'gumdrops', 'gummi', 'honey', 'icing', 'jellybeans', 'ladyfingers', 'licorice', 'macaroons', 'maple', 'marrons glaces', 'marshmallows', 'marzipan', 'molasses', 'pastries', 'pectin', 'peppermints', 'pie', 'piping', 'puddings', 'puff', 'sourball', 'sprinkles', 'sucanat', 'sugar', 'sweetener', 'syrup', 'tarts', 'toffee', 'twinkies', 'colaciones', 'sherbet', "hershey®'s", 'candies', "confectioners'", 'fudge', 'taffy', 'pink', 'sherbet']
ingredientsKB['sauces'] = ['alfredo', 'applesauce', 'chutney', 'cannoli', 'dips', 'guacamole', 'hummus', 'paste', 'spreads', 'tahini', 'tzatziki', 'denjang', 'salsa', 'sauce', 'tapenade', 'coating', 'teriyaki', 'aioli', 'checca', 'amatriciana', 'ragu', 'marinara']
ingredientsKB['condiments'] = ['dressing', 'jam', 'ketchup', 'marinade', 'marjoram', 'mayonnaise', 'mirin', 'mustard', 'pesto', 'relish', 'shoyu', 'tamari', 'vinaigrette', 'gochujang']
ingredientsKB['soups'] = ['broth', 'chowder', 'dashi', 'soup', 'stew', 'jambalaya', 'gumbo', 'gazpacho', 'goulash', 'pho', 'slumgullion', 'cioppino', 'minestrone']
ingredientsKB['nuts'] = ['almonds', 'butternuts', 'candlenuts', 'cashews', 'chestnuts', 'hazelnuts', 'macadamia', 'nuts', 'peanuts', 'pecans', 'pistachios', 'walnuts', 'nuts']
ingredientsKB['alcoholicIngredients'] = ['anisette', 'beer', 'bitters', 'bourbon', 'brandy', 'cacao', 'chambord', 'champagne', 'cognac', 'eggnog', 'kirsch', 'kirschwasser', 'liqueur', 'rum', 'schnapps', 'sherry', 'ale', 'spritz', 'tequila', 'vermouth', 'vodka', 'whiskey', 'wine', 'campari', 'alcohol', 'absinthe', 'cachaca', 'liquor', 'cointreau', 'curacao', 'sake', 'sec', 'calvados', 'galliano', 'lillet', 'margaritas', 'coladas', 'negroni', 'mojitos', 'mimosas', 'bahama', 'slammer', 'sauvignon', 'chablis', 'martinis', 'tequinis', 'spritzs', 'cosmopolitan', 'hurricanes', 'sangria', 'sex', "shaggy's", 'nipples', 'stoli']
ingredientsKB['spices'] = ['allspice', 'anise', 'arrowroot', 'basil', 'bay', 'capers', 'caraway', 'cardamom', 'cassava', 'cayenne', 'chocolate', 'cilantro', 'cinnamon', 'cloves', 'cocoa', 'coriander', 'cumin', 'dill', 'fennel', 'flax', 'garlic', 'ginger', 'herbs', 'kalonji', 'mace', 'masala', 'miso', 'monosodium', 'nutmeg', 'oregano', 'paprika', 'pepper', 'peppercorns', 'pimento', 'poppy', 'poppyseed', 'powder','rhubarb', 'rosemary', 'saffron', 'sage', 'salt', 'savory', 'seasoning', 'sesame', 'spices', 'sunflower', 'tarragon', 'thyme', 'turmeric', 'vanilla', 'watercress', 'spearmint', 'comfort']
ingredientsKB['spicy'] = ['angelica', 'dijon', 'horseradish', 'jerk', 'wasabi', 'spicy']
ingredientsKB['hotPeppers'] = ['jalapenos', 'pepperoncinis', 'chiles']
ingredientsKB['grains'] = ['bagels', 'baguettes', 'barley', 'biscuits', 'bran', 'bread', 'buns', 'cereal', 'corn', 'cornbread', 'cornstarch', 'couscous', 'crackers', 'croutons', 'crusts', 'dough', 'granola', 'hominy', 'kasha', 'masa', 'matzo', 'millet', 'muffins', 'oats', 'pitas', 'popcorn', 'pretzels', 'quinoa', 'rice', 'rolls',  'shortbread', 'sourdough', 'stuffing', 'tapioca', 'toasts', 'tortillas', 'wheat', 'kaiser', 'cornmeal', 'breadcrumbs', 'graham', 'bulgur', 'farina', 'oatmeal', 'croissants', 'polenta', 'grits', 'pumpernickel', 'sago', 'seitan', 'grains', 'taters', 'risotto', 'shells', 'amarettini', 'mochi', 'cornflakes', 'pilaf', 'puppies']
ingredientsKB['pastas'] = ['farfalle', 'fettuccine', 'lasagnas', 'linguine', 'mac', 'macaroni', 'manicotti', 'noodles', 'pasta', 'farfel', 'vermicelli', 'tagliatelle', 'cannelloni', 'penne']
ingredientsKB['wrappedMeals'] = ['burritos', 'calzones', 'dumplings', 'empanadas', 'fajitas', 'hero', 'pie', 'pinwheels', 'pizzas', 'quesadillas', 'sandwiches', 'tacos', 'tourtiere', 'wontons', 'hoagie', 'pierogies', 'rarebit', 'joes', 'enchiladas', 'pierogi', 'bierrocks', 'torta', 'reuben', 'wraps', 'piroshki', 'tamales', 'bruschetta', 'antipasto', 'hamburger', 'muffuletta', 'blanket', 'runzas', 'samosas', 'sambousas', 'chalupas', 'spanakopita', 'submarine']
ingredientsKB['pastaDishes'] = ['casseroles', 'curry', 'lasagna', 'marzetti', 'mostaccioli', 'spaghetti', 'stroganoff', 'ziti', 'pastini', 'pastitsio', 'fideo', 'spaghettini', 'moussaka', 'tortellinis', 'tallerine', 'talerine', 'scampi', 'ravioli', 'pad', 'gnocchi', 'spaetzle', 'stromboli']
ingredientsKB['vegetableDishes'] = ['tabbouleh', 'kabobs', 'suey', 'frittatas', 'quiches', 'raita', 'shieldzini', 'stir', 'sukiyaki']
ingredientsKB['drinks'] = ['beverage', 'cider', 'coffee', 'dew™', 'drink', 'eggnog', 'epazote', 'espresso', 'gin', 'juices', 'lemonade', 'limeade', 'milk', 'rosewater', 'soda', 'tea', 'wassail', 'punch', 'shake', 'shirley', 'americano']
ingredientsKB['cookingLiquids'] = ['oil', 'vinegar', 'water', 'snow', 'ice']
ingredientsKB['bakingIngredients'] = ['ammonia', 'baking', 'eggs', 'flour', 'margarine', 'yeast', 'bisquick®']
ingredientsKB['cookingFats'] = ['butter', 'gelatin', 'gravy', 'lard', 'lecithin', 'ovalette', 'shortening', 'xanthan', 'suet']
ingredientsKB['extras'] = ['carnations', 'coloring', 'dust', 'flowers', 'lilies', 'spray', 'toppings', 'drippings', 'powdered', 'gold']
ingredientsKB['fasteners'] = ['sticks', 'skewers', 'toothpicks']
ingredientsKB['adhesives'] = ['glue']
ingredientsKB['containers'] = ['jars']
ingredientsKB['flavorings'] = ['extract', 'flavorings', 'mint', 'pandan', 'hickory', 'flavored', 'mesquite', 'wood', 'hardwood']
ingredientsKB['mixtures'] = ['food', 'mixes']

# words with succeeding noun ("milk" or "cake")
ingredientsKB['nonDairyMilks'] = ['almond', 'soy', 'coconut']
ingredientsKB['cakeTypes'] = ['pound', 'sponge', 'white', 'yellow', 'bunny', "'scratch'"]

In [11]:
cuisine = ['https://www.allrecipes.com/recipes/1789/world-cuisine/european/italian/']

In [12]:
write_urls(cuisine, "json_created_files/cuisine_links.json")

In [14]:
urls = [("json_created_files/cuisine_links.json","json_created_files/cuisine")]

In [15]:
for run in urls:
    file = run[0]
    name = run[1]

    # Create Individual URL List
    with open(file, 'r') as infile:
        ind_url_list = json.load(infile)

    # Potentially remove duplicates

    for ind_url in ind_url_list:
        main_parse(ind_url,check="false",url_name="json_created_files/cuisine")

In [39]:
def clean_text(s):
    tokens = word_tokenize(s)
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    words = [word for word in stripped if word.isalpha()]
    
    return words

In [139]:
ingredients_df = pd.read_json('json_created_files/cuisineingredients.json')
ingredients_df = ingredients_df.rename(index=str, columns={0:'ingredient'})

In [140]:
ingredients_df

,ingredient
0,"romaine lettuce - torn, washed and dried"
1,torn escarole
2,torn radicchio
3,torn red leaf lettuce
4,chopped green onions
5,"red bell pepper, sliced into rings"
6,"green bell pepper, sliced in rings"
7,cherry tomatoes
8,grapeseed oil
9,chopped fresh basil


In [21]:
methods_df = pd.read_json('json_created_files/cuisinecooking.json')

In [22]:
tools_df = pd.read_json('json_created_files/cuisineutensils.json')

In [81]:
kb = [item for sublist in ingredientsKB.values() for item in sublist] 

In [141]:
ingredients = {}

for category in ingredientsKB:
    counter = Counter()
    for idx, row in ingredients_df.iterrows():
        tokens = clean_text(row['ingredient'])
        for i in tokens:
            if i in ingredientsKB[category]:
                counter[i] += 1
    if counter:
        ingredients[category] = counter

In [142]:
ingredients

{'dairyIngredients': Counter({'buttermilk': 1, 'cream': 6, 'milk': 7}),
 'cheeses': Counter({'mozzarella': 2}),
 'meats': Counter({'chops': 1, 'veal': 1, 'bacon': 2, 'ham': 1, 'livers': 1}),
 'fish': Counter({'tilapia': 1}),
 'mainProteins': Counter({'beans': 5, 'pine': 1, 'nuts': 1}),
 'vegetables': Counter({'lettuce': 2,
          'escarole': 2,
          'radicchio': 1,
          'onions': 4,
          'tomatoes': 11,
          'parsley': 10,
          'potatoes': 3,
          'peppers': 2,
          'celery': 5,
          'chard': 1,
          'leaves': 6,
          'olives': 2,
          'cucumber': 1,
          'vegetable': 3,
          'spinach': 5,
          'carrots': 3,
          'asparagus': 1,
          'mushrooms': 1,
          'cabbage': 1,
          'cilantro': 1}),
 'sugars': Counter({'sugar': 8}),
 'sauces': Counter({'paste': 3, 'sauce': 5}),
 'condiments': Counter({'marjoram': 1}),
 'soups': Counter({'broth': 6}),
 'nuts': Counter({'pecans': 2, 'nuts': 1}),
 'alcoholi

In [187]:
cuisine_df = pd.read_json('train.json')

In [188]:
italian_df = cuisine_df[cuisine_df['cuisine'] == 'italian']

In [194]:
italian = []

for i in italian_df['ingredients'].tolist():
    for j in i:
        italian += (j.lower().split())

In [195]:
italian

['sugar',
 'pistachio',
 'nuts',
 'white',
 'almond',
 'bark',
 'flour',
 'vanilla',
 'extract',
 'olive',
 'oil',
 'almond',
 'extract',
 'eggs',
 'baking',
 'powder',
 'dried',
 'cranberries',
 'chopped',
 'tomatoes',
 'fresh',
 'basil',
 'garlic',
 'extra-virgin',
 'olive',
 'oil',
 'kosher',
 'salt',
 'flat',
 'leaf',
 'parsley',
 'pimentos',
 'sweet',
 'pepper',
 'dried',
 'oregano',
 'olive',
 'oil',
 'garlic',
 'sharp',
 'cheddar',
 'cheese',
 'pepper',
 'swiss',
 'cheese',
 'provolone',
 'cheese',
 'canola',
 'oil',
 'mushrooms',
 'black',
 'olives',
 'sausages',
 'italian',
 'parsley',
 'leaves',
 'walnuts',
 'hot',
 'red',
 'pepper',
 'flakes',
 'extra-virgin',
 'olive',
 'oil',
 'fresh',
 'lemon',
 'juice',
 'trout',
 'fillet',
 'garlic',
 'cloves',
 'chipotle',
 'chile',
 'fine',
 'sea',
 'salt',
 'flat',
 'leaf',
 'parsley',
 'fresh',
 'parmesan',
 'cheese',
 'butter',
 'all-purpose',
 'flour',
 'fat',
 'free',
 'less',
 'sodium',
 'chicken',
 'broth',
 'chopped',
 'fresh'

In [203]:
ingredients = {}

in_kb = set()

for category in ingredientsKB:
    counter = Counter()
    for i in italian:
        if i in ingredientsKB[category]:
            counter[i] += 1
            in_kb.add(i)
    if counter:
        ingredients[category] = counter.most_common()

In [200]:
not_kb = Counter()

for i in italian:
    if i not in in_kb:
        not_kb[i] += 1

In [202]:
not_kb.most_common()

[('cheese', 5843),
 ('fresh', 4589),
 ('olive', 4566),
 ('ground', 2981),
 ('black', 2473),
 ('parmesan', 2456),
 ('grated', 2069),
 ('red', 2053),
 ('chicken', 1825),
 ('dried', 1687),
 ('chopped', 1514),
 ('lemon', 1401),
 ('extra-virgin', 1362),
 ('large', 1291),
 ('onion', 1228),
 ('dry', 1210),
 ('italian', 1127),
 ('juice', 970),
 ('all-purpose', 919),
 ('tomato', 873),
 ('crushed', 870),
 ('leaf', 729),
 ('bell', 725),
 ('green', 692),
 ('kosher', 666),
 ('ricotta', 665),
 ('cooking', 641),
 ('flat', 597),
 ('shredded', 591),
 ('beef', 583),
 ('unsalted', 580),
 ('flakes', 527),
 ('egg', 527),
 ('diced', 513),
 ('whole', 451),
 ('sausage', 450),
 ('sodium', 415),
 ('boneless', 406),
 ('freshly', 403),
 ('minced', 388),
 ('fat', 379),
 ('heavy', 377),
 ('balsamic', 372),
 ('plum', 366),
 ('purple', 351),
 ('skinless', 347),
 ('sea', 340),
 ('zucchini', 340),
 ('frozen', 339),
 ('part-skim', 335),
 ('low', 326),
 ('reggiano', 319),
 ('crumbs', 314),
 ('parmigiano', 308),
 ('pizza'

In [204]:
ingredients

{'dairyIngredients': [('cream', 994),
  ('milk', 739),
  ('cottage', 107),
  ('buttermilk', 48),
  ('yogurt', 34),
  ('creme', 4),
  ('creamer', 1),
  ('creamy', 1),
  ('half-and-half', 1)],
 'cheeses': [('mozzarella', 1249),
  ('swiss', 84),
  ('bocconcini', 17),
  ('gouda', 15),
  ('brie', 14)],
 'meats': [('prosciutto', 257),
  ('bacon', 231),
  ('ribs', 132),
  ('veal', 129),
  ('sausages', 115),
  ('ham', 97),
  ('chops', 44),
  ('meatballs', 33),
  ('sirloin', 27),
  ('livers', 14),
  ('hash', 6),
  ('chorizo', 6),
  ('mortadella', 5),
  ('rounds', 3),
  ('brisket', 2),
  ('dogs', 2),
  ('meats', 1),
  ('mignon', 1),
  ('mincemeat', 1),
  ('venison', 1),
  ('tripe', 1)],
 'poultry': [('turkey', 186),
  ('salami', 68),
  ('chickens', 10),
  ('hens', 5),
  ('quail', 3),
  ('wings', 2)],
 'fish': [('salmon', 32),
  ('fish', 31),
  ('bass', 15),
  ('halibut', 14),
  ('tilapia', 10),
  ('snapper', 8),
  ('flounder', 5),
  ('whitefish', 4),
  ('grouper', 3),
  ('catfish', 3),
  ('sole'